In [1]:
import pandas as pd

In [2]:
dataset=pd.read_csv("input/dataset_final.csv")

This are all the columns we have to play with. I will try to make my best to create the fastest and simplest algorithm, trying to infere the keyword "desempleo_y" along time. From the practical point of view, this is quite non sense, I am trying to face a time series problem with the cheapest and fastest regression I can.

I am well aware of this situation, but the goal is learning a lot about:
- Feature selection and engineering
- Otimising my ML algorithm
- Perform by hand many of the tasks left to ml libraries
- Using several Cloud Functions within an automated pipeline
- Keep learning about Google Cloud tools
- Get a ML result within a low RAM Cloud Function

In [3]:
list(dataset.columns)

['Unnamed: 0',
 'date',
 'protestas_x',
 'extremismo_x',
 'rebelion_x',
 'refugiados_x',
 'precio_petroleo_x',
 'juicio_x',
 'corrupcion_x',
 'inestabilidad_politica_x',
 'terrorismo_x',
 'vigilancia_x',
 'seguridad_nacional_x',
 'ejercito_x',
 'protestas_y',
 'extremismo_y',
 'rebelion_y',
 'refugiados_y',
 'precio_petroleo_y',
 'juicio_y',
 'corrupcion_y',
 'inestabilidad_politica_y',
 'terrorismo_y',
 'vigilancia_y',
 'seguridad_nacional_y',
 'ejercito_y',
 'stock_market',
 'quiebra_economica',
 'inflacion_economica',
 'precio_petroleo',
 'libre_comercio',
 'job_quality_&_labor_market_performance',
 'banco_mundial',
 'finanzas_y_bancos',
 'desempleo_x',
 'prosperidad_economica_y_finanzas',
 'incertidumbre_economica',
 'crecimiento_economico',
 'macroeconomia_deuda_y_vulnerabilidad',
 'pobreza',
 'ciencia',
 'racismo',
 'inmigracion_x',
 'emergencia_sanitaria',
 'vacunas',
 'censura_en_medios',
 'emprendimiento_x',
 'energias_renovables',
 'subsidios',
 'agresion_sexual',
 'precio_vi

In [4]:
dataset.drop(columns='Unnamed: 0', inplace=True)
# Unemployment, which is how much people searches "desempleo" in Spain, will be my target to infere
dataset.rename(columns={'desempleo_y':"unemployment"}, inplace=True)

# Sliding time

- I'm going to create a new dataset with *unemployment*, my target column and date
- Remove *unemployment* from the old dataset
- In the old dataset, add 3 weeks to the date rows. In this way, everythin happening right now will be linked with unemployment 3 weeks ahead
- Merge the *unemployment* and date dataset with the old one with changed dates

Summing up, change the dates 3 weeks ahead of everything but *unemployment* column

In [5]:
dataset.tail()

,date,protestas_x,extremismo_x,rebelion_x,refugiados_x,precio_petroleo_x,juicio_x,corrupcion_x,inestabilidad_politica_x,terrorismo_x,...,taxi,teletrabajo,tinder,uber,uber eats,videoconferencia,videollamada,vox,yoga,zoom
89,2020-09-20,-7.955000,-4.300,-4.933333,-3.205,0.0,-44.481429,-53.120000,-7.48,-3.300000,...,29,100,88,45,75,17,7,8,47,22
90,2020-09-27,-10.885714,-4.375,-2.670000,-3.600,0.0,-57.642857,-28.765714,0.00,-7.586000,...,29,43,88,48,90,17,8,10,46,20
91,2020-10-04,-15.368000,-3.420,0.000000,-3.880,0.0,-73.910000,-18.664286,-39.07,-4.770000,...,29,24,82,46,91,10,5,8,45,20
92,2020-10-11,-6.241667,0.000,-1.250000,-4.330,0.0,-39.984286,-30.545714,-14.97,-3.846667,...,27,38,90,47,100,8,7,11,42,18
93,2020-10-18,-8.295000,0.000,-4.800000,-0.665,0.0,-103.931667,-36.138571,-3.99,-5.055000,...,26,25,82,47,99,13,7,11,39,21


In [6]:
len(dataset.columns)

132

In [7]:
dataset.date

0     2019-01-06
1     2019-01-13
2     2019-01-20
3     2019-01-27
4     2019-02-03
         ...    
89    2020-09-20
90    2020-09-27
91    2020-10-04
92    2020-10-11
93    2020-10-18
Name: date, Length: 94, dtype: object

In [8]:
# i thought on doing this but i think the path described is more efficient
#df_slide=dataset[dataset.columns.difference(['unemployment'])]
#dataset.loc[:, ~dataset.columns.isin(['unemployment'])]
dataset["date"]=pd.to_datetime(dataset["date"])
df_unemployment=pd.DataFrame(dataset[["date", "unemployment"]])
dataset.drop(columns="unemployment", inplace=True)

In [9]:
df_unemployment

,date,unemployment
0,2019-01-06,23
1,2019-01-13,19
2,2019-01-20,18
3,2019-01-27,21
4,2019-02-03,23
...,...,...
89,2020-09-20,34
90,2020-09-27,41
91,2020-10-04,39
92,2020-10-11,30


In [10]:
from datetime import timedelta

In [11]:
dataset["date"]=dataset["date"].apply(lambda x: x+timedelta(weeks=3))
#dataset["date"]=dataset["date"] + timedelta(weeks=3)

In [12]:
dataset

,date,protestas_x,extremismo_x,rebelion_x,refugiados_x,precio_petroleo_x,juicio_x,corrupcion_x,inestabilidad_politica_x,terrorismo_x,...,taxi,teletrabajo,tinder,uber,uber eats,videoconferencia,videollamada,vox,yoga,zoom
0,2019-01-27,-0.986667,-29.993333,-21.453333,-6.425000,0.00,-111.880000,-92.164000,-7.770000,-82.140000,...,42,2,56,27,10,2,2,35,50,5
1,2019-02-03,-16.754286,-2.260000,-17.413333,-15.266667,0.00,-485.394286,-168.828571,-23.500000,-11.586667,...,49,1,51,35,15,3,1,27,44,4
2,2019-02-10,-98.830000,-5.644000,-26.305000,0.000000,3.72,-232.251429,-106.468571,-36.120000,-145.260000,...,100,2,50,100,9,3,2,16,41,3
3,2019-02-17,-41.448571,-4.226667,-26.570000,1.276000,0.00,-233.122857,-106.342857,-24.146667,-34.084000,...,85,2,52,78,11,3,3,11,49,4
4,2019-02-24,-35.362857,0.000000,-28.106667,-20.310000,0.00,-173.451429,-112.497143,-5.160000,-24.512000,...,67,1,50,40,11,3,2,13,48,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
89,2020-10-11,-7.955000,-4.300000,-4.933333,-3.205000,0.00,-44.481429,-53.120000,-7.480000,-3.300000,...,29,100,88,45,75,17,7,8,47,22
90,2020-10-18,-10.885714,-4.375000,-2.670000,-3.600000,0.00,-57.642857,-28.765714,0.000000,-7.586000,...,29,43,88,48,90,17,8,10,46,20
91,2020-10-25,-15.368000,-3.420000,0.000000,-3.880000,0.00,-73.910000,-18.664286,-39.070000,-4.770000,...,29,24,82,46,91,10,5,8,45,20
92,2020-11-01,-6.241667,0.000000,-1.250000,-4.330000,0.00,-39.984286,-30.545714,-14.970000,-3.846667,...,27,38,90,47,100,8,7,11,42,18


In [13]:
# now let's merge again
df=pd.merge(dataset,df_unemployment,how='outer', on=["date"],suffixes=(None,None))
df.sort_values(by=["date"],inplace=True)
df=df.fillna(0)

In [14]:
df.reset_index(drop=True,inplace=True)

In [15]:
df.drop([0,1,2], inplace=True)

In [16]:
df

,date,protestas_x,extremismo_x,rebelion_x,refugiados_x,precio_petroleo_x,juicio_x,corrupcion_x,inestabilidad_politica_x,terrorismo_x,...,teletrabajo,tinder,uber,uber eats,videoconferencia,videollamada,vox,yoga,zoom,unemployment
3,2019-01-27,-0.986667,-29.993333,-21.453333,-6.425000,0.00,-111.880000,-92.164000,-7.770000,-82.140000,...,2.0,56.0,27.0,10.0,2.0,2.0,35.0,50.0,5.0,21.0
4,2019-02-03,-16.754286,-2.260000,-17.413333,-15.266667,0.00,-485.394286,-168.828571,-23.500000,-11.586667,...,1.0,51.0,35.0,15.0,3.0,1.0,27.0,44.0,4.0,23.0
5,2019-02-10,-98.830000,-5.644000,-26.305000,0.000000,3.72,-232.251429,-106.468571,-36.120000,-145.260000,...,2.0,50.0,100.0,9.0,3.0,2.0,16.0,41.0,3.0,23.0
6,2019-02-17,-41.448571,-4.226667,-26.570000,1.276000,0.00,-233.122857,-106.342857,-24.146667,-34.084000,...,2.0,52.0,78.0,11.0,3.0,3.0,11.0,49.0,4.0,23.0
7,2019-02-24,-35.362857,0.000000,-28.106667,-20.310000,0.00,-173.451429,-112.497143,-5.160000,-24.512000,...,1.0,50.0,40.0,11.0,3.0,2.0,13.0,48.0,4.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
92,2020-10-11,-7.955000,-4.300000,-4.933333,-3.205000,0.00,-44.481429,-53.120000,-7.480000,-3.300000,...,100.0,88.0,45.0,75.0,17.0,7.0,8.0,47.0,22.0,30.0
93,2020-10-18,-10.885714,-4.375000,-2.670000,-3.600000,0.00,-57.642857,-28.765714,0.000000,-7.586000,...,43.0,88.0,48.0,90.0,17.0,8.0,10.0,46.0,20.0,27.0
94,2020-10-25,-15.368000,-3.420000,0.000000,-3.880000,0.00,-73.910000,-18.664286,-39.070000,-4.770000,...,24.0,82.0,46.0,91.0,10.0,5.0,8.0,45.0,20.0,0.0
95,2020-11-01,-6.241667,0.000000,-1.250000,-4.330000,0.00,-39.984286,-30.545714,-14.970000,-3.846667,...,38.0,90.0,47.0,100.0,8.0,7.0,11.0,42.0,18.0,0.0
